In [10]:
import numpy as np
import pandas as pd
from numpy.fft import fft, fftfreq, ifft
import scipy.fftpack
from matplotlib import *
import matplotlib.pyplot as plt
import scipy
from scipy import *
from scipy import signal
import mpld3
mpld3.enable_notebook()
import sys
import os
import random
import mne
from mne.time_frequency import psd_welch
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from sklearn.metrics import r2_score
from mne.decoding import CSP 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import svm
import pywt
from sklearn.model_selection import KFold
numpy.set_printoptions(threshold=sys.maxsize)

In [11]:
m2_time_stamps_beg_movement=np.array([10.21,10.12,10.79,10.09,9.9,8.22,8.94,8.28,8.25,9.21,9.58,9.5,9.79,11.87,10.18])
m2_sample_number_beg_movement=floor(m2_time_stamps_beg_movement*250)
print(m2_sample_number_beg_movement)


[2552. 2530. 2697. 2522. 2475. 2055. 2235. 2070. 2062. 2302. 2395. 2375.
 2447. 2967. 2545.]


C:\Users\mnaser1\AppData\Local\Temp\ipykernel_31956\2270942163.py:2: DeprecationWarning: scipy.floor is deprecated and will be removed in SciPy 2.0.0, use numpy.floor instead
  m2_sample_number_beg_movement=floor(m2_time_stamps_beg_movement*250)


In [12]:
# first four rows of OpenBCI were removed manually
rest_concat=np.ndarray(shape=(1,8))
directory = os.fsencode(r"C:\Users\mnaser1\OneDrive - Kennesaw State University\Desktop\PhD-S3\Research\Gait intention\M2\r")

for file in os.listdir(directory):
    filename_l = list(os.fsdecode(file))
    current_file= str(r"C:\Users\mnaser1\OneDrive - Kennesaw State University\Desktop\PhD-S3\Research\Gait intention\M2\r")+str(r"\\")+str(os.fsdecode(file))
    rest=pd.read_csv(current_file, sep=",")
    rest=rest[[" EXG Channel 0"," EXG Channel 1"," EXG Channel 2"," EXG Channel 3"," EXG Channel 4"," EXG Channel 5"," EXG Channel 6"," EXG Channel 7"]]
    rest=rest.tail(750)  # keep last 3s of recording for rest
    print("rest shape",rest.shape)
    rest_np=np.array(rest)
    rest_concat=np.concatenate((rest_concat,rest_np),axis=0)
rest_concat=np.delete(rest_concat,0,axis=0)
print("size of rest file accross all sessions is:",rest_concat.shape)


i=0
walk_concat=np.ndarray(shape=(1,8))
directory = os.fsencode(r"C:\Users\mnaser1\OneDrive - Kennesaw State University\Desktop\PhD-S3\Research\Gait intention\M2\w")
for file in os.listdir(directory):
    filename_l = list(os.fsdecode(file))
    current_file= str(r"C:\Users\mnaser1\OneDrive - Kennesaw State University\Desktop\PhD-S3\Research\Gait intention\M2\w")+str(r"\\")+str(os.fsdecode(file))
    walk=pd.read_csv(current_file, sep=",")
    walk=walk[[" EXG Channel 0"," EXG Channel 1"," EXG Channel 2"," EXG Channel 3"," EXG Channel 4"," EXG Channel 5"," EXG Channel 6"," EXG Channel 7"]]
    walk_np=np.array(walk)
    beg_here=int(m2_sample_number_beg_movement[i])
    walk_np=walk_np[:beg_here]
    beg_here2=len(walk_np)-750
    walk_np=walk_np[beg_here2:beg_here2+750]
    print("walk shape",walk_np.shape)
    walk_concat=np.concatenate((walk_concat,walk_np),axis=0)
    i=i+1
walk_concat=np.delete(walk_concat,0,axis=0)
print("size of walk file accross all sessions is:",walk_concat.shape)

full_data=np.concatenate((rest_concat,walk_concat),axis=0)
print("size of walk and rest accross all sessions is:",full_data.shape)

rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
rest shape (750, 8)
size of rest file accross all sessions is: (11250, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
walk shape (750, 8)
size of walk file accross all sessions is: (11250, 8)
size of walk and rest accross all sessions is: (22500, 8)


In [13]:
events=np.ndarray(shape=(1,3),dtype=int)
for i in range(0,len(full_data),750):
    my_index=i
    if my_index<len(walk_concat):
        my_label=1
    else:
        my_label=2
    current_event=np.array([my_index, 0 ,my_label])   
    current_event=np.reshape(current_event,(1,3))
    events=np.append(events,current_event,axis=0)
events=np.delete(events,0,axis=0)
#print(events)

In [14]:
channels=["FP1","FP2","C3","C4","F7","F8","F3","F4"]
full_data_transposed=transpose(full_data)
info=mne.create_info(ch_names=channels, sfreq=250, ch_types='eeg')
raw =mne.io.RawArray(full_data_transposed, info,1)
raw.set_eeg_reference('average')
raw.filter(7, 13, fir_design='firwin', skip_by_annotation='edge')

Creating RawArray with float64 data, n_channels=8, n_times=22500
    Range : 1 ... 22500 =      0.004 ...    90.000 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 413 samples (1.652 sec)



C:\Users\mnaser1\AppData\Local\Temp\ipykernel_31956\3010243025.py:2: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
  full_data_transposed=transpose(full_data)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


<RawArray | 8 x 22500 (90.0 s), ~1.4 MB, data loaded>

In [15]:
event_id = dict(rest=1, walk=2)
tmin, tmax = .004,3
#reject=dict(eeg=10000)
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=channels, baseline=None, preload=True,reject=None)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 750 original time points ...
0 bad epochs dropped


In [16]:
##### LDA-raw
moving_window_acc_arr=[]
epochs_data = epochs.get_data()
avg_microv=np.ndarray(shape=(30,750),dtype=float)
sec_inc=.5
sec_inc_samp=sec_inc*250
num_intervals=int(3/sec_inc)

for counter in range(1,num_intervals+1):
    s_end=750
    s_start=int(s_end-(sec_inc_samp*counter))
    for ii in range(0,30):
        for i in range (s_start,s_end):
            avg_microv[ii][i]=(epochs_data[ii][0][i]+epochs_data[ii][1][i]+epochs_data[ii][2][i]+epochs_data[ii][3][i]+epochs_data[ii][4][i]+epochs_data[ii][5][i]+epochs_data[ii][6][i]+epochs_data[ii][7][i])/8
            #avg_microv[ii][i]=(epochs_data[ii][2][i]+epochs_data[ii][3][i])/2

    lda = LinearDiscriminantAnalysis()

    my_labels=epochs.events[:, -1]
    scores = []

    cv = ShuffleSplit(4, test_size=0.20, random_state=20)
    clf =lda
    scores = cross_val_score(clf, avg_microv, my_labels, cv=cv, n_jobs=None)
    #print(scores)
    print("avergae score over cvs for",s_start,"-",s_end,"is:",np.mean(scores))
    mean_score=np.mean(scores)
    #print(scores)
    moving_window_acc_arr.append(mean_score)
#plt.plot(moving_window_acc_arr)

avergae score over cvs for 625 - 750 is: 0.7916666666666667
avergae score over cvs for 500 - 750 is: 0.7916666666666667
avergae score over cvs for 375 - 750 is: 0.75
avergae score over cvs for 250 - 750 is: 0.875
avergae score over cvs for 125 - 750 is: 0.875
avergae score over cvs for 0 - 750 is: 0.5833333333333333


In [21]:
##### svm-raw
moving_window_acc_arr=[]
epochs_data = epochs.get_data()
avg_microv=np.ndarray(shape=(30,750),dtype=float)
sec_inc=.5
sec_inc_samp=sec_inc*250
num_intervals=int(3/sec_inc)

for counter in range(1,num_intervals+1):
    s_end=750
    s_start=int(s_end-(sec_inc_samp*counter))
    for ii in range(0,30):
        for i in range (s_start,s_end):
            avg_microv[ii][i]=(epochs_data[ii][0][i]+epochs_data[ii][1][i]+epochs_data[ii][2][i]+epochs_data[ii][3][i]+epochs_data[ii][4][i]+epochs_data[ii][5][i]+epochs_data[ii][6][i]+epochs_data[ii][7][i])/8
            #avg_microv[ii][i]=(epochs_data[ii][2][i]+epochs_data[ii][3][i])/2

    my_labels=epochs.events[:, -1]
    scores = []

    cv = ShuffleSplit(4, test_size=0.15, random_state=20)
    clf =svm.SVC(kernel='linear',C=100, gamma=.1)    
    scores = cross_val_score(clf, avg_microv, my_labels, cv=cv, n_jobs=None)
    #print(scores)
    print("avergae score over cvs for",s_start,"-",s_end,"is:",np.mean(scores))
    mean_score=np.mean(scores)
    #print(scores)
    moving_window_acc_arr.append(mean_score)
#plt.plot(moving_window_acc_arr)

avergae score over cvs for 625 - 750 is: 0.95
avergae score over cvs for 500 - 750 is: 0.95
avergae score over cvs for 375 - 750 is: 0.95
avergae score over cvs for 250 - 750 is: 0.95
avergae score over cvs for 125 - 750 is: 0.95
avergae score over cvs for 0 - 750 is: 0.2
